# Piecewise Conversion

Model equipment with **load-dependent efficiency** using piecewise linear approximation.

**User Story:** A gas engine's efficiency varies with load - lower at part-load, optimal at mid-load. We want to capture this non-linear behavior in our optimization.

In [ ]:
import numpy as np
import pandas as pd

import flixopt as fx

fx.CONFIG.notebook()

## The Problem

Real equipment efficiency varies with operating point:

| Load Level | Electrical Efficiency | Reason |
|------------|----------------------|--------|
| 25-50% (part load) | 32-38% | Throttling losses |
| 50-75% (mid load) | 38-42% | Near design point |
| 75-100% (full load) | 42-40% | Thermal limits |

A constant efficiency assumption misses this behavior.

## Define the Efficiency Curve

Each `Piece` defines corresponding fuel input and electricity output ranges:

In [ ]:
piecewise_efficiency = fx.PiecewiseConversion(
    {
        'Gas': fx.Piecewise(
            [
                fx.Piece(start=78, end=132),  # Part load
                fx.Piece(start=132, end=179),  # Mid load
                fx.Piece(start=179, end=250),  # Full load
            ]
        ),
        'Electricity': fx.Piecewise(
            [
                fx.Piece(start=25, end=50),  # 32% -> 38% efficiency
                fx.Piece(start=50, end=75),  # 38% -> 42% efficiency
                fx.Piece(start=75, end=100),  # 42% -> 40% efficiency
            ]
        ),
    }
)

## Build and Solve

In [ ]:
timesteps = pd.date_range('2024-01-22', periods=24, freq='h')

# Demand varies through the day (30-90 kW, within piecewise range 25-100)
elec_demand = 60 + 30 * np.sin(np.arange(24) * np.pi / 12)

fs = fx.FlowSystem(timesteps)
fs.add_elements(
    fx.Bus('Gas'),
    fx.Bus('Electricity'),
    fx.Effect('costs', '€', is_standard=True, is_objective=True),
    fx.Port('GasGrid', imports=[fx.Flow(bus='Gas', size=300, effects_per_flow_hour=0.05)]),
    fx.Converter(
        'GasEngine',
        inputs=[fx.Flow(bus='Gas')],
        outputs=[fx.Flow(bus='Electricity')],
        piecewise_conversion=piecewise_efficiency,
    ),
    fx.Port('Load', exports=[fx.Flow(bus='Electricity', size=1, fixed_relative_profile=elec_demand)]),
)

fs.optimize(fx.solvers.HighsSolver());

## Visualize the Efficiency Curve

In [ ]:
fs.components['GasEngine'].piecewise_conversion.plot(x_flow='Gas')

## Results

In [ ]:
fs.stats.plot.balance('Electricity')

In [ ]:
# Verify efficiency varies with load
fuel = fs.solution['GasEngine(Gas)|flow_rate']
elec = fs.solution['GasEngine(Electricity)|flow_rate']
efficiency = elec / fuel

print(f'Efficiency range: {float(efficiency.min()):.1%} - {float(efficiency.max()):.1%}')
print(f'Total cost: {fs.solution["costs"].item():.2f} €')

## Key Points

**Syntax:**
```python
fx.PiecewiseConversion({
    'Input': fx.Piecewise([fx.Piece(start=a, end=b), ...]),
    'Output': fx.Piecewise([fx.Piece(start=x, end=y), ...]),
})
```

**Rules:**
- All flows must have the **same number of segments**
- Segments typically **connect** (end of N = start of N+1)
- Efficiency = output / input at each point

**Time-varying:** Pass arrays instead of scalars to model changing limits (e.g., temperature derating).

**Next:** See [06c-piecewise-effects](06c-piecewise-effects.ipynb) for non-linear investment costs.